# Comparison star generator

In [ ]:
import re
import requests

import astropy.units as u
from astropy.coordinates import SkyCoord

from stellarphot import SourceListData
from stellarphot.gui_tools.comparison_functions import ComparisonViewer

## Change the settings in this cell as needed

In [ ]:
bright_mag_limit = 9
dim_mag_limit = 16
Cmag = 10

# 👉 File to save apertures in
aperture_output_file = 'aperture_locations.csv'

# 👉 File to save SourceListData in
source_list_file = 'source_list.ecsv'

# 👉 File to exoplanet info in
tess_info_output_file = 'tess-info.pickle'

In [ ]:
comp_viewer = ComparisonViewer(directory="..",
                           target_mag=Cmag,
                           bright_mag_limit=bright_mag_limit,
                           dim_mag_limit=dim_mag_limit,
                           targets_from_file=None,
                           aperture_output_file=aperture_output_file,
                           overwrite_outputs=True
                          )
comp_viewer.box

In [ ]:
import pickle

with open(tess_info_output_file, 'wb') as f:
    pickle.dump(comp_viewer.toi_info, f)
    

In [ ]:
##
## Convert output aperture file to new SourceListData object format and resave
##

targets_table = comp_viewer.generate_table()
# Assign units to columns
targets_table['ra'] = targets_table['ra'] * u.deg
targets_table['dec'] = targets_table['dec'] * u.deg
targets_table['x'] = targets_table['x'] * u.pixel
targets_table['y'] = targets_table['y'] * u.pixel
# Drop redundant sky position column
targets_table.remove_columns(['coord'])
# Build sources list
targets2sourcelist = {'x' : 'xcenter', 'y' : 'ycenter'}
sources = SourceListData(input_data=targets_table, colname_map=targets2sourcelist)

# Save to ECSV file
sources.write(source_list_file, format='ascii.ecsv', overwrite=True)